In [15]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

In [16]:
file_path = r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\txt files\dotabuff_match_history.txt"
# Open the file in binary mode
with open(file_path, "rb") as file:
    # Read the binary content of the file
    binary_content = file.read()
decoded_content = binary_content.decode("utf-8", errors="replace")

soup = BeautifulSoup(decoded_content, 'html.parser')


In [17]:
# Create Dataframe if df does not already exist
columns = ['MatchID', 'Date', 'Hero','Match Duration', 'Result', 'Kills', 'Deaths', 'Assists', 'Networth', 'Team']

if 'df' not in locals():
    df = pd.DataFrame(columns=columns)
    first_index = 0
    
    for i in range(1,6):                                            # Separate for loops to have Enemy_pos 1-5, then display Ally_pos 1-5 after
        df['Enemy_pos' + str(i)] = ""

    for i in range(1,6):
        if i != 5: # Do not include myself as the 5th teammate
            df['Ally_pos' + str(i)] = ""

       
else:
    df = pd.read_csv(r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\dota.csv")
    first_index = first_index + 50

In [18]:
columns_add = ['MatchID', 'Date', 'Hero','Match Duration', 'Result', 'Kills', 'Deaths', 'Assists']

In [19]:
df_add_data = pd.DataFrame(columns = columns_add)

In [21]:
# Find all match tags 
heroes = soup.find_all("a", href=lambda href: href and '/matches/72' in href)

# Within each match tag, take info of hero I played, matchid, and match result from html
hero_tags = soup.find_all('a', href=lambda href: href and '/matches/' in href)

matchids = []
heroes = []
results = []

# Add data into the matchids, heroes, results lists
for tag in hero_tags:

    # Extract hero name if the tag has "class" as an attribute
    if not tag.has_attr("class"):
        heroes.append(tag.text)

        # Extract the contents under the "href" attribute then split by "/" then take the MatchID
        matchid = tag['href'].split('/')[2]
        matchids.append(matchid)

     # Tags that don't have "class" attribute include the result of the match
    else:
        results.append(tag.text)

In [22]:
for i in range(0, 50):
          df_add_data.loc[i, 'Hero'] = heroes[i]
          df_add_data.loc[i, 'Result'] = results[i]
          df_add_data.loc[i, 'MatchID'] = matchids[i]

In [23]:
for i in range(0, 50):
   # Find the matchid of the row
   current_match_id = df_add_data.loc[i,"MatchID"]

   # Find Starting <a> tag for each matchid
   starting_a_tag = soup.find_all('a', href=lambda href: href and '/matches/' + current_match_id in href)[1]  
   
   # Find the Date of the match
   date_tag_content = starting_a_tag.find_next('time')['title']
   date_time_format = datetime.strptime(date_tag_content, "%a, %d %b %Y %H:%M:%S %z")
   date = datetime.strftime(date_time_format, "%Y-%m-%d")
   df_add_data.loc[i, "Date"] = date

   # Find the duration of each match
   duration = starting_a_tag.find_all_next('td')[1].text  # Find the duration by finding the second <td> tag
   df_add_data.loc[i, 'Match Duration'] = duration  # Insert duration into row
   
   # Find the KDA (Kills/Deaths/Assists) of each match
   kda = starting_a_tag.find_all_next('td')[2].text  # Find the Kills/Deaths/Assists of the match
   df_add_data.loc[i, 'Kills'] = kda.split('/')[0]
   df_add_data.loc[i, 'Deaths'] = kda.split('/')[1]
   df_add_data.loc[i, 'Assists'] = kda.split('/')[2]

In [24]:
# Add the new data to the existing dataframe

df = pd.concat([df,df_add_data], axis = 0 )

In [26]:
csv_dota2 = 'dota.csv'
df.to_csv(csv_dota2, index=False)


In [27]:
df.tail()

,MatchID,Date,Hero,Match Duration,Result,Kills,Deaths,Assists,Networth,Team,Enemy_pos1,Enemy_pos2,Enemy_pos3,Enemy_pos4,Enemy_pos5,Ally_pos1,Ally_pos2,Ally_pos3,Ally_pos4
45,7218834860,2023-06-27,Undying,25:55,Won Match,6,1,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,7217551342,2023-06-27,Undying,44:11,Lost Match,7,10,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,7216278139,2023-06-26,Venomancer,48:43,Lost Match,13,6,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,7216256731,2023-06-26,Venomancer,30:23,Won Match,4,2,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,7216236208,2023-06-26,Techies,30:10,Lost Match,2,8,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
